# Build a RAG pipeline to generate Telegram Amharic Ad Posts

build RAG pipeline for amharic language. 
- Retrive the relevant information from english and amharic texts about 
    - Advertiser information
    - Product being advertised
    - Campaign inputs i.e. briefs

- Craft a good prompt 
- Generate Candidate Ads
- Evaluate Pipeline

## Import Packages

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.vectorstores import Weaviate

from typing import List, Optional, Union

import weaviate
from weaviate.embedded import EmbeddedOptions

### Add the embedding

In [5]:

# embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embedding_model_name = "iocuydi/llama-2-amharic-3784m"
# embedding_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

model_kwargs = {}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name, 
  model_kwargs=model_kwargs
)

/home/babi/miniconda3/envs/tenx_week7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/babi/.cache/torch/sentence_transformers/iocuydi_llama-2-amharic-3784m. Creating a new one with MEAN pooling.
/home/babi/miniconda3/envs/tenx_week7/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/babi/miniconda3/envs/tenx_week7/lib/python3.11/site-packages/torch/cuda/__init__.py:740: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch versio

OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory /home/babi/.cache/torch/sentence_transformers/iocuydi_llama-2-amharic-3784m.

In [24]:

# embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
# emedding_model_name = "iocuydi/llama-2-amharic-3784m"
embedding_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"

model_kwargs = {}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name, 
  model_kwargs=model_kwargs
)

.gitattributes: 100%|██████████| 1.35k/1.35k [00:00<00:00, 7.05MB/s]
README.md: 100%|██████████| 2.15k/2.15k [00:00<00:00, 16.1MB/s]
config.json: 100%|██████████| 617/617 [00:00<00:00, 4.00MB/s]
merges.txt: 100%|██████████| 15.4M/15.4M [00:11<00:00, 1.39MB/s]
model.safetensors: 100%|██████████| 1.77G/1.77G [03:15<00:00, 9.05MB/s]
pytorch_model.bin: 100%|██████████| 1.77G/1.77G [03:05<00:00, 9.55MB/s]
training_args.bin: 100%|██████████| 2.29k/2.29k [00:00<00:00, 15.5MB/s]
vocab.json: 100%|██████████| 19.5M/19.5M [00:02<00:00, 8.32MB/s]
No sentence-transformers model found with name /home/babi/.cache/torch/sentence_transformers/uhhlt_am-roberta. Creating a new one with MEAN pooling.
Some weights of RobertaModel were not initialized from the model checkpoint at /home/babi/.cache/torch/sentence_transformers/uhhlt_am-roberta and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [18]:
def data_loader(file_path: str, chunk_size: int = 500, chunk_overlap: int = 50) -> Union[List[str], None]:
    """
    Load data from a file, split it into chunks, and return the chunks.

    Parameters:
    - file_path (str): The path to the file containing the data.
    - chunk_size (int): The size of each data chunk. Default is 500.
    - database (int): The overlap between consecutive chunks. Default is 50.

    Returns:
    - list: A list of data chunks.
    """
    try:
        loader = TextLoader(file_path)
        documents = loader.load()

        # Chunk the data
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        chunks = text_splitter.split_documents(documents)
        
        print("Data loaded to vector database successfully")
        return chunks
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None 
    
    
def create_retriever(chunks):
   try:
    #    Setup vector database
       client = weaviate.Client(embedded_options=EmbeddedOptions())

       # Populate vector database using embeddings from the Hugging Face model
       vectorstore = Weaviate.from_documents(
           client=client,
           documents=chunks,
           embedding=embeddings,  # Use the model's encode function for embeddings
           by_text=False
       )

       # Define vectorstore as retriever to enable semantic search
       retriever = vectorstore.as_retriever()
       print("Retriever created successfully.")

       return retriever

   except Exception as e:
       print(f"An unexpected error occurred: {e}")
       return None

## Load the data and create chunks

In [5]:
chuncks = data_loader("../prompts/context.txt")
len(chuncks)

Data loaded to vector database successfully


11

In [6]:
retriever = create_retriever(chuncks)

/home/babi/miniconda3/envs/tenx_week7/lib/python3.11/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(
{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-01T10:18:08+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-01T10:18:08+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-01T10:18:08+03:00"}


Started /home/babi/.cache/weaviate-embedded: process ID 30294


{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-02-01T10:18:08+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-02-01T10:18:08+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"2024-02-01T10:18:08+03:00"}
{"level":"info","msg":"Completed loading shard langchain_094829e85d36449ab3b283e1ac2fbf04_vqcZmuLhfJ9y in 9.853602ms","time":"2024-02-01T10:18:09+03:00"}
{"level":"info","msg":"Completed loading shard langchain_698f357610544f228bc230b3c3c1c248_yHzTxe2uDT3I in 3.264151ms","time":"2024-02-01T10:18:09+03:00"}
{"level":"info","msg":"Completed loading shard langchain_0312ef7773c64bd5a9d3d2ba96f090fb_0TLKGeLxTnEG in 11.097574ms","time":"2024-02-01T10:18:09+03:00"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info

Retriever created successfully.


In [7]:
type(retriever)

langchain.vectorstores.base.VectorStoreRetriever

In [12]:
retriever.get_relevant_documents("ማሳሰቢያ")

[Document(page_content='ማሳሰቢያ : ውጤት ማይት የሚቻለው ከቀኑ 8 ሰኣት ጀምሮ ነው። የላካቹ የ ቤተሰቦች እንደየቀደም ተከተላችሁ ውጤቱን\nከ8 ሰኣት ጀምሮ እንልክላችዋለን። በተቻለን አቅም ኔትዎርክ ባላችሁበት አካባቢ ለሚያስቸግራቹ ውጤታችሁን ለማየትና\nወደ እናተ ለማድረስ እንሞክራለን።\nየ12ኛ ክፍል ብሄራዊ ፈተና ውጤት ይፋ ሆነ በድረ ገፅ ለመከታተል ይጠቀሙ። በ መልእክት () ለመከታተል 8181 ላይ\nበማስቀደም የምዝገባ ቁጥር ( ) በማስገባት ና በመላክ ውጤት ማወቅ ይቻላል። መልካሙ ሁሉ ኢዲገጥማችሁ ይመኛል !\nብትልኩልን ውጤቱን እንልክላቹዋለን ! በተለይ ኔትዋርክ የሚያስቸግራቹ !\nውጤት እያደረስኳቹ ነው ግን ብዙ ሰዎችን እያስተናገድኩ ስለሆነ ከላካቹ በሃላ ቢያንስ 3 ደቂቃ ታገሱ ማድረስ ይቻላል !\nቀጥሉ ያላያቹ ብቻ ! ላኩ በአንድ ጊዜ ከአንድ በላይ እንዳትልኩ !\nቀጣይ ዙር ! ብቻ ላኩ', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ሆኖም የኢትዮጵያ ታሪክ ሊያስተምረን እንደሚችለው ኢትዮጵያዊነት \nእንደ ላስቲክ ነው። ሲስቡት ይሳባል፣ሲለቁት ይሰበሰባል። ማለት ይሳብና ይሳሳል እንጂ አይበጠስም። ፕሮ መስፍን ወ/ማርያም\nሰዉ ዉበቱን እንጂ ቁንጅናዉን ሊፈጥር አይችልም:: ተፈጥሮ የሰጠንን አጣጥበን \nከመዉጣት በዘለለ ቁንጅናን ልንፈጥረዉ አንችልም:: ማቆንጀት የሚገባን ጭንቅላታችንን ነዉ:: ደስ የሚል ቀን ይሁንላችሁ !\nሀይማኖት የግል ነው ሀገር የጋራ ነው ቁልፉና ማሰሪያው ኢትዮጵያዊነት ነው\nበ24 ሰኣት ውስጥ 400 ኢትዮጵያዊ አባላትን አግኝተናል እናመሰግናለን !', metadata={'source': '../prompts/context.txt'}),
 Docume

In [14]:
retriever.get_relevant_documents("ተማሪዎች")

[Document(page_content='በ5ኛው ዙር ለ 100 ተማሪዎች ውጤታቸውን አይቼ ልካለሁ ብቻ\nየመጨረሻ 250 ሰው አስተናግዳለሁ : ሀገሪቱ ውስጥ ባለው የኔትወርክ መጨናነቅ ምክንያት ውጤት ማየት ያልቻላችሁ \nብቻ ወደ ላኩ እስከ ምሽቱ 12:30 ድረስ! አመሰግናለሁ !\nበቅድሚያ እንኳን ደስ አላችሁ ለማለት እወዳለሁ ውድ የሀገሬ ልጆች ! ወደ እኔ ተልከው ያየዋቸው ውጤቶች ደስ\nየሚያሰኙ ናቸው ምንም እንኳን ከአምናው ጋር ሲነፃፀር ዝቅተኛ ቢሆንምነገር ግን የመግቢያው ነጥብ ዝቅ የሚል \nይመስለኛል ምክንያቱ ደግሞ በሀገሪቱ ውስጥ የተከፈቱት 11 አዳዲስ ዩኒቨርሲቲዎች የተማሪዎችን ቅበላ ከፍ ስለሚያደርገው።\nካየዋቸው በመቶዎች ከሚቆጠሩ ውጤቶች መካከል ከፍተኛው 625 ነው። ይቅርታ ! ተሳስቼ የሌላ ሰው ልኬላችሁ ከሆነ ይቅርታ !\nበጣም ዘግይቼ ልኬላችሁ ከሆነ ሁላችሁም እንኳን ደስ አላችሁ ! ደስታቹ ደስታችን ነው !', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ማሳሰቢያ : ውጤት ማይት የሚቻለው ከቀኑ 8 ሰኣት ጀምሮ ነው። የላካቹ የ ቤተሰቦች እንደየቀደም ተከተላችሁ ውጤቱን\nከ8 ሰኣት ጀምሮ እንልክላችዋለን። በተቻለን አቅም ኔትዎርክ ባላችሁበት አካባቢ ለሚያስቸግራቹ ውጤታችሁን ለማየትና\nወደ እናተ ለማድረስ እንሞክራለን።\nየ12ኛ ክፍል ብሄራዊ ፈተና ውጤት ይፋ ሆነ በድረ ገፅ ለመከታተል ይጠቀሙ። በ መልእክት () ለመከታተል 8181 ላይ\nበማስቀደም የምዝገባ ቁጥር ( ) በማስገባት ና በመላክ ውጤት ማወቅ ይቻላል። መልካሙ ሁሉ ኢዲገጥማችሁ ይመኛል !\nብትልኩልን ውጤቱን እንልክላቹዋለን ! በተለይ ኔትዋርክ የሚያስቸግራቹ !\nውጤት እያደረስኳቹ ነው ግን ብዙ ሰዎችን እያስተናገድ

In [23]:
retriever.get_relevant_documents("ኢትዮጵያ")

[Document(page_content='ሆኖም የኢትዮጵያ ታሪክ ሊያስተምረን እንደሚችለው ኢትዮጵያዊነት \nእንደ ላስቲክ ነው። ሲስቡት ይሳባል፣ሲለቁት ይሰበሰባል። ማለት ይሳብና ይሳሳል እንጂ አይበጠስም። ፕሮ መስፍን ወ/ማርያም\nሰዉ ዉበቱን እንጂ ቁንጅናዉን ሊፈጥር አይችልም:: ተፈጥሮ የሰጠንን አጣጥበን \nከመዉጣት በዘለለ ቁንጅናን ልንፈጥረዉ አንችልም:: ማቆንጀት የሚገባን ጭንቅላታችንን ነዉ:: ደስ የሚል ቀን ይሁንላችሁ !\nሀይማኖት የግል ነው ሀገር የጋራ ነው ቁልፉና ማሰሪያው ኢትዮጵያዊነት ነው\nበ24 ሰኣት ውስጥ 400 ኢትዮጵያዊ አባላትን አግኝተናል እናመሰግናለን !', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ኢትዮጵያዊነት ኢትዮጵያዊነት ብዙ የተለያዩ ህብረተሰቦች የተዋህዱበት አካል ክፍል\nመሆን ነው። ኢትዮጵያዊነት ከጎሰኝኛት በላይና ውጭ የሆነ አጠቃላይ ማንነት ነው።\nኢትዮጵያዊነት የዚች ውጥንቅጥ መሬት ባለቤትነት ነው፣ ኢትዮጵያዊነት የደጋው\nብርድና የቆላው ሙቀት የሚገናኙበት፣ የደጋው ዝናብም ከቆላው ወንዝ ጋር\n\nየተዛመደበት ሃይል ነው። ኢትዮጵያዊነት ለሙሴ፣ ለክርስቶስና ለመሀመድ የሚጨሰው \nእጣን በእርገት መጥቶ የሚደባለቅበት እምነት ነው። ኢትዮጵያዊነት በቄጤማ ጉዝጓዝ\nላይ የሚታየው መተሳሰብና መፈቃቀር ነው። ኢትዮጵያዊነት የአስተዋይነት፣የሚዛናዊነት \nየጨዋነት ባሕርይ ነው። ኢትዮጵያዊነት ረዢምና ተፅፎ ያላለቀ ታሪክ ነው፤ ብዙ ሰዎችም\nየተሰዉለት ስሜት ነው።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ኢትዮጵያዊነት ስቃይና መከራ ነው፤ ኢትዮጵያዊነት ኩራትና\nክብርም ነው። ኢትዮጵያዊነት 